# 과제 진행을 위한 사전 준비

##### psycopg2 설치

In [1]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


##### PostgreSQL - Python 연동

In [2]:
import psycopg2
connection = psycopg2.connect("host=49.50.167.136 dbname=synthea_1000 user= walker102 password=forcebewithyou port=5432")

In [3]:
cur = connection.cursor()

In [4]:
# test
cur.execute("SELECT person_id from person")
print(cur.fetchone())

(402435,)


# 문제 풀이
## 1번 문제 
visit_occurrence에서 모든 환자에 대한 내원 일수 구한 수 내원 일수의 최댓값과 그 환자를 찾기 

In [5]:
cur.execute("""with temp as ( 
                    select person_id, 
                           sum(vo.visit_end_date - vo.visit_start_date +1) as visit_cnt
                    from visit_occurrence as vo
                    group by person_id)
                select person_id, 
                       visit_cnt
                from temp
                where visit_cnt = (select max(visit_cnt) from temp);""")
# person_id, visit_cnt
print(cur.fetchone()) 

(1059760, 18873)


## 2번 문제
환자들이 진단받은 병명 중에 a,b,c,d,e 로 시작하고 중간에 heart가 포함된 것 찾기
,코드도 함께

In [6]:
cur.execute("""with temp as (
                    select co.condition_concept_id, 
                           c.concept_name 
                    from concept as c
                    join condition_occurrence as co 
                      on c.concept_id = co.condition_concept_id
                    where concept_name ~* '^a|^b|^c|^d|^e')
            select temp.condition_concept_id, 
                   temp.concept_name
            from temp
            where concept_name like '%heart%'
            group by condition_concept_id, 
                     concept_name;""")
# condition_concept_id, concept_name
print(cur.fetchone()) 

(4229440, 'Chronic congestive heart failure')


## 3번 문제
drug_exposure에서 '1891866'환자의 처방된 약 종류별로 처음 시작일, 마지막 종료일, 복용일(enddate-startdate+1)을 구하고 복용일이 긴 순으로 정렬하여 테이블 생성

###### 자료를 넣을 테이블을 생성하기 위한 SQL쿼리문
create table drug_take (drug_concept_id int, take_start_date timestamp, take_end_date timestamp, take_cnt int);

In [7]:
# connection.rollback()
cur.execute("""select drug_concept_id,
                      min(date(drug_exposure_start_datetime)) as take_start_date,
                      max(date(drug_exposure_end_datetime)) as take_end_date,
                      sum(date(drug_exposure_end_datetime) - date(drug_exposure_start_datetime) + 1) as take_cnt
               from drug_exposure
               where person_id = '1891866'
               group by drug_concept_id
               order by take_cnt desc;""")
#drug_taken_data = cur.fetchall()
drug_take_data = cur.fetchall()
print(drug_take_data)

[(19009384, datetime.date(1959, 12, 1), datetime.date(1998, 10, 6), 14424), (1539463, datetime.date(1990, 3, 13), datetime.date(1998, 3, 11), 5484), (19030765, datetime.date(1988, 10, 18), datetime.date(1998, 10, 5), 1214), (40213154, datetime.date(1989, 9, 12), datetime.date(1998, 7, 7), 10), (40213227, datetime.date(1993, 1, 5), datetime.date(1993, 1, 5), 1)]


##### 아래 셀 실행하면 테이블에 데이터가 들어갑니다!(이미 4번 들어갔습니다..)

In [30]:
insert_query = """insert into drug_take(drug_concept_id, take_start_date, take_end_date, take_cnt) values(%s,%s,%s,%s)"""

for data in drug_take_data:
    print(data)
    cur.execute(insert_query, data)
connection.commit()

(19009384, datetime.date(1959, 12, 1), datetime.date(1998, 10, 6), 14424)
(1539463, datetime.date(1990, 3, 13), datetime.date(1998, 3, 11), 5484)
(19030765, datetime.date(1988, 10, 18), datetime.date(1998, 10, 5), 1214)
(40213154, datetime.date(1989, 9, 12), datetime.date(1998, 7, 7), 10)
(40213227, datetime.date(1993, 1, 5), datetime.date(1993, 1, 5), 1)


## (PASS) 4번 문제


## 5번 문제
* 방문마다 다른지 / 사람마다 다른지, 90일이 연속적인지 / 분할인지, 중복 여부는? (한 환자가 두 번 방문해서 둘 다 90일 이상 복용했다면?)

* 방문마다 다른 건수로 카운트, 쪼개진 90일 복용은 의미 없을거라 생각해 90일이 연속적으로 나올 경우만 고려, 더불어 person_id 중복도 카운트에 포함 (한 환자가 두번 90번 복용했어도 다른 건수로 카운트 해야하지 않는가?)

In [8]:
cur.execute("""with da as (
                    select p.person_id, p.year_of_birth, 
                           co.condition_concept_id, 
                           co.visit_occurrence_id
                    from person as p
                    join condition_occurrence as co 
                    on p.person_id =co.person_id
                    where co.condition_concept_id in (3191208,36684827,3194332,3193274,43531010,4130162,45766052,45757474,4099651,4129519,4063043,4230254,4193704,4304377,201826,3194082,3192767) 
                    and p.year_of_birth < 2005),
                med as (
                    select de.person_id, 
                           de.drug_concept_id, 
                           de.drug_exposure_start_datetime, 
                           de.drug_exposure_end_datetime,
                           date(de.drug_exposure_end_datetime) - date(de.drug_exposure_start_datetime) + 1 as taken_cnt
                    from drug_exposure as de
                    join da 
                    on de.person_id = da.person_id
                    where drug_concept_id = '40163924' 
                    and date(de.drug_exposure_end_datetime) - date(de.drug_exposure_start_datetime) + 1 > 89)
                select count(person_id)
                from med;""")
# count
print(cur.fetchone()) 

(690,)


## (PASS) 6번 문제

## (미완성) 7번 문제
clinical_note는 한 환자의 의료 기록

* insertPerson() : person 테이블에 넣을 데이터를 뽑아오는 함수

In [9]:
def insertPerson(s):
    personid,year,month,day,deathd = 0,0,0,0,0 # 아이디, 년, 월, 일, 사망날짜
    gender,race,ethnicity = '','','' # 성별, 인종, 민족성
    data = s.split('\n') # 줄바꿈으로 나누기
    # 데이터 가져오기
    for i in range(2,len(data)): 
        temp = data[i].split(':')
        if temp[0] == 'Race':
            race = temp[1].replace(" ","")
        elif temp[0] == 'Ethnicity':
            ethnicity = temp[1].replace(" ","")
        elif temp[0] == 'Gender':
            gender = temp[1].replace(" ","")
        elif temp[0] == 'Birth Date':
            bdate = temp[1].replace(" ","").split('-')
            year, month, day = bdate[0],bdate[1],bdate[2]
    print(race, ethnicity, gender, year, month, day)

In [10]:
cur.execute("""select note
            from clinical_note;""")
data_ = cur.fetchone()

data = data_[0].split('--------------------------------------------------------------------------------')
# 위처럼 나눴을 때 len(data) == 5 이므로, 0,1,2까지만 뽑으면 COUNTINUING 이후 제외됨
for i in range(3):
    print(data[i])
    print('*************')
        
print(insertPerson(data[0]))
    
    

Andrea7 Wolf938
Race:                White
Ethnicity:           Non-Hispanic
Gender:              M
Age:                 55
Birth Date:          1965-04-22
Marital Status:      M

*************

ALLERGIES:
No Known Allergies

*************

ENCOUNTER
2011-06-20 : Encounter at Cape Cod Vet Center : Encounter for Acute bronchitis (disorder)
Type: ambulatory
   
   MEDICATIONS:
  2011-06-20 : Acetaminophen 325 MG Oral Tablet for Acute bronchitis (disorder)
   
   CONDITIONS:
  2011-06-20 : Acute bronchitis (disorder)
   
   CARE PLANS:
  2011-06-20 : Respiratory therapy
                         Reason: Acute bronchitis (disorder)
                         Activity: Recommendation to avoid exercise
                         Activity: Deep breathing and coughing exercises
   
   REPORTS:
   
   OBSERVATIONS:
   
   PROCEDURES:
  2011-06-20 : Sputum examination (procedure) for Acute bronchitis (disorder)
   
   IMMUNIZATIONS:
   
   IMAGING STUDIES:
   

*************
White Non-Hispanic M 1965